<a href="https://colab.research.google.com/github/soedinglab/spacepharer/blob/master/examples/SpacePHARER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SpacePHARER: CRISPR Spacer Phage-Host pAiRs findER
---

<img src="https://raw.githubusercontent.com/soedinglab/spacepharer/master/.github/SpacePHARER.png" height="192" align="right" style="height:192px"/>

SpacePHARER is a modular toolkit for sensitive phage-host interaction identification using CRISPR spacers. SpacePHARER adapts the fast homology search capabilities of [MMseqs2](https://github.com/soedinglab/MMseqs2) to sensitively query short spacer sequences. It introduces a novel approach of aggregating sets of spacer-based hits to discover phage-host matches.

[Zhang, R., Mirdita, M., Levy Karin, E., Norroy, C., Galiez, C., & Söding, J.  SpacePHARER: Sensitive identification of phages from CRISPR spacers in prokaryotic hosts. Bioinformatics, doi:10.1093/bioinformatics/btab222 (2021).](https://doi.org/10.1093/bioinformatics/btab222)



In [ ]:
#@title Set parameters, hit `Runtime` -> `Run all`, wait for file input box to appear below this cell to upload either 1) all your spacer files or 2) your genome files
jobname = 'test' #@param {type:"string"}
#@markdown ---
#@markdown Input options (input mode is ignored if example data is used)
input_mode = "CRISPR spacers" #@param ["CRISPR spacers", "Phage genomes"]
use_example_data = False #@param {type:"boolean"}
#@markdown ---
#@markdown Advanced options
fdr_cutoff = 0.05 #@param {type:"number"}
#@markdown ---
#@markdown Don't forget to hit `Runtime` -> `Run all` after updating the form. An input box will appear below this text. Upload your spacers or genomes.

jobname = "".join(jobname.split())

input_type = 0
if input_mode == "CRISPR spacers":
  input_type = 1

import os
import shutil
shutil.rmtree(jobname + "_input", ignore_errors=True)
os.makedirs(jobname + "_input", exist_ok=True)

if use_example_data:
  input_type = 1
  import urllib.request
  urllib.request.urlretrieve("https://raw.githubusercontent.com/soedinglab/spacepharer/32ff17396bd5b91f4a68bbdede4f7806c6c6ad44/examples/JNGQ01000029.fas", os.path.join(jobname + "_input", "JNGQ01000029.fas"))
  urllib.request.urlretrieve("https://raw.githubusercontent.com/soedinglab/spacepharer/32ff17396bd5b91f4a68bbdede4f7806c6c6ad44/examples/CP003088.fas", os.path.join(jobname + "_input", "CP003088.fas"))
else:
  from google.colab import files
  uploaded = files.upload()
  for fn in uploaded.keys():
    os.replace(fn, os.path.join(jobname + "_input", fn))


In [ ]:
#@title Download dependencies and databases
%%bash -s $input_type
INPUT_TYPE=$1
if [ ! -e SPACEPHARER_READY ]; then
  wget -q https://github.com/soedinglab/spacepharer/releases/download/5-c2e680a/spacepharer-linux-avx2.tar.gz
  tar -xzf spacepharer-linux-avx2.tar.gz
  rm -f spacepharer-linux-avx2.tar.gz
  touch SPACEPHARER_READY
fi

if [ ! -e SPACEPHARER_DB_READY ] && [ "${INPUT_TYPE}" = "1" ]; then
  mkdir -p database
  spacepharer/bin/spacepharer downloaddb GenBank_phage_2018_09 database/gb1809 tmp -v 0 &> /dev/null
  touch SPACEPHARER_DB_READY
fi

if [ ! -e SPACEPHARER_SPACERS_READY ] && [ "${INPUT_TYPE}" = "0" ]; then
  mkdir -p database
  spacepharer/bin/spacepharer downloaddb spacers_shmakov_et_al_2017 database/shmakov17 tmp -v 0 &> /dev/null
  touch SPACEPHARER_SPACERS_READY
fi

In [ ]:
#@title Run SpacePHARER
%%bash -s $jobname $fdr_cutoff $input_type
JOBNAME=$1
FDR=$2
INPUT_TYPE=$3
if [ "$INPUT_TYPE" = "1" ]; then
  spacepharer/bin/spacepharer easy-predict "${JOBNAME}_input/"* database/gb1809 "$JOBNAME" tmp --fdr $FDR -v 0 &> /dev/null
else
  if [ ! -e "database/${JOBNAME}_db.dbtype" ]; then
    spacepharer/bin/spacepharer createsetdb "${JOBNAME}_input/"* database/${JOBNAME}_db tmp -v 0 &> /dev/null
    spacepharer/bin/spacepharer createsetdb "${JOBNAME}_input/"* database/${JOBNAME}_db_rev tmp --reverse-fragments 1 -v 0 &> /dev/null
  fi
  spacepharer/bin/spacepharer predictmatch database/shmakov17 database/${JOBNAME}_db database/${JOBNAME}_db_rev "$JOBNAME" tmp --fdr $FDR -v 0 &> /dev/null
fi

# compute taxonomy results
if [ -e ${JOBNAME}_lca.tsv ]; then
  cat ${JOBNAME}_lca.tsv | awk '{$1 = NR; print;}' > ${JOBNAME}_lca_tmp.tsv
  spacepharer/bin/spacepharer tsv2db ${JOBNAME}_lca_tmp.tsv ${JOBNAME}_lca_tmp_db --output-dbtype 8 -v 0
  spacepharer/bin/spacepharer taxonomyreport database/gb1809_nucl_orf ${JOBNAME}_lca_tmp_db ${JOBNAME}_lca.html --report-mode 1 -v 0
  spacepharer/bin/spacepharer rmdb ${JOBNAME}_lca_tmp_db -v 0
  rm -f -- ${JOBNAME}_lca_tmp.tsv
fi
if [ -e ${JOBNAME}_lca_per_target.tsv ]; then
  cat ${JOBNAME}_lca_per_target.tsv | awk '{$1 = NR; print;}' > ${JOBNAME}_lca_tmp.tsv
  spacepharer/bin/spacepharer tsv2db ${JOBNAME}_lca_tmp.tsv ${JOBNAME}_lca_tmp_db --output-dbtype 8 -v 0
  spacepharer/bin/spacepharer taxonomyreport database/shmakov17_nucl_orf ${JOBNAME}_lca_tmp_db ${JOBNAME}_lca_per_target.html --report-mode 1 -v 0
  spacepharer/bin/spacepharer rmdb ${JOBNAME}_lca_tmp_db -v 0
  rm -f -- ${JOBNAME}_lca_tmp.tsv
fi

In [ ]:
#@title Show taxonomic distribution with Krona
from IPython.display import HTML
from IPython.core.display import display
from pathlib import Path
suffix = "_lca.html" if input_type == 1 else "_lca_per_target.html"
html = '<script>onload()</script></html>'.join(Path(jobname + suffix).read_text().rsplit('</html>', 1))
display(HTML(html))

In [ ]:
#@title Show spacer hits
from pathlib import Path
from IPython.display import HTML
from IPython.core.display import display
from pathlib import Path

html = '''
<style>
tbody tr.head {
  font-weight: bold;
}
</style>
<table style='text-align:left'>
  <thead>
    <tr>
      <th>Prokaryote</th>
      <th>Phage</th>
      <th>Comb. score</th>
      <th colspan="5">Count</th>
    </tr>
    <tr>
      <th colspan='2'>Spacer</th>
      <th>P-val</th>
      <th>Spacer start</th>
      <th>End</th>
      <th>Phage start</th>
      <th>End</th>
      <th>5'_PAM|3'_PAM</th>
      <th>PAM rev. strand</th>
    </tr>
  </thead>
<tbody>
'''

for line in Path('test').read_text().split('\n'):
  if len(line) == 0:
    continue
  if line[0] == '#':
    cols = line.split('\t')
    if len(cols) == 4:
      html += "<tr class='head'>"
      html += "<td>" + cols[0][1:] + "</td>"
      if input_type == 1:
        html += "<td><a target='_blank' rel='noopener' href='https://www.ncbi.nlm.nih.gov/nuccore/" + cols[1] + "'>" + cols[1] + "</a></td>"
      else:
        html += "<td>" + cols[1] + "</td>"
      html += "<td>" + cols[2] + "</td>"
      html += "<td colspan='6'>" + cols[3] + "</td>"
      html += "</tr>\n"
  elif line[0] == '>':
    cols = line.split('\t')
    if len(cols) == 9:
      html += "<tr>"
      html += "<td colspan='2'>" + cols[0][1:] + "</td>"
      html += "<td>" + cols[2] + "</td>"
      html += "<td>" + cols[3] + "</td>"
      html += "<td>" + cols[4] + "</td>"
      html += "<td>" + cols[5] + "</td>"
      html += "<td>" + cols[6] + "</td>"
      html += "<td>" + cols[7] + "</td>"
      html += "<td>" + cols[8] + "</td>"
    html += "</tr>\n"
html += '''
</tbody>
</table>
'''
display(HTML(html))


In [ ]:
#@title Download result files as zip
suffix = "lca" if input_type == 1 else "lca_per_target"
!zip -FSr $jobname".result.zip" $jobname $jobname"_"$suffix".tsv" $jobname"_"$suffix".html"
from google.colab import files
files.download(f"{jobname}.result.zip")